# Sentece Piece 

- unsupervised text tokenizer and detokenizer 이며 주로 딥러닝에서 사용
- 내부적으로 BPE (byte-pair-encoding) 그리고 unigram language model 을 사용
- 특정 언어에 국한되지 않고, 다양한 언어에 사용 가능

- [논문](https://arxiv.org/pdf/1808.06226.pdf)
- [Github](https://github.com/google/sentencepiece)

설치

```
$ sudo apt-get install cmake build-essential pkg-config libgoogle-perftools-dev
$ pip install sentencepiece
```

In [4]:
from pathlib import Path
from tempfile import gettempdir
from typing import Dict, List

import numpy as np
import pandas as pd
import requests
import sentencepiece as stp
from konlpy.tag import Okt

okt = Okt()

# Data

In [8]:
def download(url, filename, force=False):
    path = Path(gettempdir()) / filename
    if not path.exists() or force:
        with open(path, "wt") as f:
            r = requests.get(url, allow_redirects=True)
            f.write(r.text)

    df = pd.read_csv(path, delimiter="\t")
    return path, df


def preprocess_morph(text) -> str:
    morphs = okt.pos(str(text))
    tokens = []
    for word, pos in morphs:
        if pos in ("Punctuation", "Foreign"):
            continue

        tokens.append(word)
    return " ".join(tokens)


train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
test_url = "https://github.com/e9t/nsmc/raw/master/ratings_test.txt"

train_path, train_df = download(train_url, "nsmc_train.txt")
test_path, test_df = download(test_url, "nsmc_test.txt")

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
train_df["morph"] = train_df.document.apply(preprocess_morph)
test_df["morph"] = test_df.document.apply(preprocess_morph)

print("train_path:", train_path)
print("test_path :", test_path)
print(f"train_df  : {train_df.shape}")
print(f"test_df   : {test_df.shape}")
test_df.sample(5)

train_path: /tmp/nsmc_train.txt
test_path : /tmp/nsmc_test.txt
train_df  : (149995, 4)
test_df   : (49997, 4)


,id,document,label,morph
29288,7796626,"""아이언맨3 보고난후 제목이 연관되어서 보게됐는데 """"아이언"""" 들어가는 영화는 대...",1,아이언맨 3 보고 난후 제목 이 연관 되어서 보게 됐는데 아이언 들어가는 영화 는 ...
76,8366483,쇼타군 넘 좋아 돈키호테 대박 재밌음,1,쇼타 군 넘 좋아 돈키호테 대박 재밌음
39568,7554622,그럭저럭 맹숭하다.,0,그럭저럭 맹숭하 다
34664,8098518,"1편의 빈 디젤에 비해서 2편의 아이스 큐브는 너무나 약해보이고, 액션도 전혀 인상...",0,1 편의 빈 디젤 에 비 해서 2 편의 아이스 큐브 는 너무나 약 해보이고 액션 도...
8048,8349698,♥예수님사랑해요오오오오,1,예수님 사랑 해 요 오오오오


# Sentencepiece with text file

## Train with text File 
- input: 학습 파일 위치
- model_prefix: 모델이름
- vocab_size: vocabulary 단어 크기
- model_type: `unigram` (default) | `bpe` | `char` | `word`
- max_sentence_length: 문장 최대 길이
- pad_id: pad token ID
- unk_id: unknown token ID
- bos_id: Begin of sentence token ID
- eos_id: End of sentence token ID 
- user_defined_symbols: 사용자 정의 토큰

In [9]:
train_morph_path = Path(gettempdir()) / "sentencepiece-train.txt"
model_prefix_path = Path(gettempdir()) / "nsmc-sentencepiece"
train_df.morph.to_csv(train_morph_path, index=False, header=False)

stp.SentencePieceTrainer.train(
    input=train_morph_path,
    model_prefix=model_prefix_path,
    vocab_size=4000,
    user_defined_symbols=["foo", "bar"],
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /tmp/sentencepiece-train.txt
  input_format: 
  model_prefix: /tmp/nsmc-sentencepiece
  model_type: UNIGRAM
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: foo
  user_defined_symbols: bar
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_dif

# Inference

In [14]:
sp = stp.SentencePieceProcessor()
sp.load(str(model_prefix_path.with_suffix(".model")))


text = test_df.sample().morph.values[0]
print('Text             :', text)
print('Encode          :', sp.Encode(text))
print('Encode as IDs   :', sp.EncodeAsIds(text))
print('Encode as Pieces:', sp.EncodeAsPieces(text))
print('Decode from IDs :', sp.decode(sp.Encode(text)))

Text             : 극장 포스터 에 손수건 을 갖고오세요 라고 써있던게 기억나네요
Encode          : [638, 885, 10, 765, 89, 206, 11, 1126, 15, 238, 1515, 258, 963, 409, 131, 30, 261, 46, 111]
Encode as IDs   : [638, 885, 10, 765, 89, 206, 11, 1126, 15, 238, 1515, 258, 963, 409, 131, 30, 261, 46, 111]
Encode as Pieces: ['▁극장', '▁포스터', '▁에', '▁손', '수', '건', '▁을', '▁갖', '고', '오', '세요', '▁라고', '▁써', '있', '던', '게', '▁기억', '나', '네요']
Decode from IDs : 극장 포스터 에 손수건 을 갖고오세요 라고 써있던게 기억나네요


In [13]:
text_list = test_df.sample(3).morph.tolist()

print('[Text]')
display(text_list)


print('\n[Encode]')
encoded = sp.encode(text_list)
print(encoded)

print('\n[Encode as Pieces]')
print([sp.encode_as_pieces(line) for line in text_list])

print('\n[Decode]')
sp.decode(encoded)

[Text]


['드라마 의 감동 을 깨지 않으려면 영화 는 보지마라',
 '보느니 후레쉬맨 보는게 낫다 그래픽 특수분장 완죤 티 난다',
 '썩 괜찮은 영화 인디 영화 의 매력 이란 게 이런거 아니겠어 잊을수 없는 조중동 디스 ㅋㅋㅋㅋㅋㅋㅋ']


[Encode]
[[73, 8, 78, 11, 1067, 18, 551, 2181, 109, 7, 14, 2810], [157, 1707, 115, 331, 458, 1425, 869, 1580, 1613, 1676, 2903, 237, 230, 1774, 3288, 1013, 1404], [2370, 915, 7, 38, 377, 7, 8, 352, 701, 72, 2109, 272, 625, 45, 5, 2284, 149, 89, 108, 187, 369, 388, 709, 61, 2685]]

[Encode as Pieces]
[['▁드라마', '▁의', '▁감동', '▁을', '▁깨', '지', '▁않', '으려', '면', '▁영화', '▁는', '▁보지마라'], ['▁보', '느', '니', '▁후', '레', '쉬', '맨', '▁보는게', '▁낫다', '▁그래픽', '▁특수', '분', '장', '▁완', '죤', '▁티', '▁난다'], ['▁썩', '▁괜찮은', '▁영화', '▁인', '디', '▁영화', '▁의', '▁매력', '▁이란', '▁게', '▁이런거', '▁아니', '겠', '어', '▁', '잊', '을', '수', '▁없는', '▁조', '중', '동', '▁디', '스', '▁ᄏᄏᄏᄏᄏᄏᄏ']]

[Decode]


['드라마 의 감동 을 깨지 않으려면 영화 는 보지마라',
 '보느니 후레쉬맨 보는게 낫다 그래픽 특수분장 완죤 티 난다',
 '썩 괜찮은 영화 인디 영화 의 매력 이란 게 이런거 아니겠어 잊을수 없는 조중동 디스 ᄏᄏᄏᄏᄏᄏᄏ']